In [1]:
article = '''Mumbai: Analysts tracking Bajaj Finance reiterated their bullish outlook after the Pune-based non-bank lender 's first-quarter business growth update took the Street by surprise, prompting brokerage houses to upgrade their recommendations and raise their respective price targets. Shares of Bajaj Finance rose as high as 8% to post their biggest single-day gain in nearly one year. The stock surged to near two-year high on Tuesday to close at ₹7,868 apiece on the NSE, up 7.3% from the previous close, extending its run of gains to six consecutive sessions. CLSA raised EPS estimates by 5-6% and upgraded the stock to buy. The global brokerage expects the stock to rise another 15% from the current levels."Bajaj Finance reported very strong pre-quarter numbers for Q1FY24," said CLSA in a client note. "While expected 6-7% QoQ AUM growth, the company delivered 9%. New customer acquisition healthy and volume growth in disbursements a strong 34% YoY."Morgan Stanley raised its price target which has the potential to deliver another 18% returns from the current levels."AUM growth of 9.2% QoQ, 32% YoY with value & vol growth & strong customer acquisition should dispel loan growth debate," Morgan Stanley told clients in a note. "Bajaj Finance has a strong credit track record; RoA is at a historical high. See this large liquid stock re-rating to 30x F25e P/E."Bajaj Finance's new loans booked during Q1 grew by 34% to 9.94 million as compared to 7.42 million in the corresponding quarter of the previous year. Assets under management (AUM) grew 32% YoY - the highest-ever quarterly increase - to ₹2.7 lakh crore.The customer franchise stood at 72.98 million at the end of June 2023 as compared to 60.30 million as of June 2022 after seeing the highest-ever quarterly increase.BofA Securities also raised its price target. It said the first quarter update should also reinforce confidence in its long-term growth guidance of 25-27%. "The company is on track to beat the upper end of its 11-12 million customer acquisition guidance comfortably," the firm said.'''

In [2]:
!pip install transformers

# 1. GPT2

In [3]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2LMHeadModel

In [4]:


class GPT2ArticleClassifier:
    def __init__(self):
        self.model_name = 'gpt2'
        self.model = GPT2ForSequenceClassification.from_pretrained(self.model_name)
        self.tokenizer = GPT2Tokenizer.from_pretrained(self.model_name)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.labels = ['adverse', 'growth']
        self.model.to(self.device)

    def summarize_article(self, article, max_length=100):
        model_name = 'gpt2'
        model = GPT2LMHeadModel.from_pretrained(model_name)
        tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        inputs = tokenizer.encode(article, return_tensors='pt')
        inputs = inputs.to(device)

        with torch.no_grad():
            outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1)
            summarized_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return summarized_text

    def classify_theme(self, article):
        inputs = self.tokenizer(article, return_tensors='pt')
        inputs = inputs.to(self.device)

        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            predicted_label_id = torch.argmax(logits, dim=1).item()
            predicted_label = self.labels[predicted_label_id]
            growth_probability = probabilities[0][1].item()
            adverse_probability = probabilities[0][0].item()

        print("Growth Probability:", growth_probability)
        print("adverse Probability:", adverse_probability)

        return predicted_label




In [5]:
# Example usage
article = article

classifier = GPT2ArticleClassifier()
predicted_label = classifier.classify_theme(article)
print("Predicted Label:", predicted_label)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Growth Probability: 0.10013208538293839
adverse Probability: 0.8998678922653198
Predicted Label: adverse


In [6]:
summarized_text = classifier.summarize_article(article)
print("Summarized Text:", summarized_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 477, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Summarized Text: Mumbai: Analysts tracking Bajaj Finance reiterated their bullish outlook after the Pune-based non-bank lender's first-quarter business growth update took the Street by surprise, prompting brokerage houses to upgrade their recommendations and raise their respective price targets. Shares of Bajaj Finance rose as high as 8% to post their biggest single-day gain in nearly one year. The stock surged to near two-year high on Tuesday to close at ₹7,868 apiece on the NSE, up 7.3% from the previous close, extending its run of gains to six consecutive sessions. CLSA raised EPS estimates by 5-6% and upgraded the stock to buy. The global brokerage expects the stock to rise another 15% from the current levels."Bajaj Finance reported very strong pre-quarter numbers for Q1FY24," said CLSA in a client note. "While expected 6-7% QoQ AUM growth, the company delivered 9%. New customer acquisition healthy and volume growth in disbursements a strong 34% YoY."Morgan Stanley raised its price

# 2. Roberta

In [7]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

class RobertaArticleClassifier:
    def __init__(self):
        self.model_name = 'roberta-base'
        self.model = RobertaForSequenceClassification.from_pretrained(self.model_name)
        self.tokenizer = RobertaTokenizer.from_pretrained(self.model_name)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.labels = ['adverse', 'growth']
        self.model.to(self.device)

    def classify_theme(self, article):
        inputs = self.tokenizer.encode_plus(article, return_tensors='pt', truncation=True, padding=True)
        inputs = inputs.to(self.device)

        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            predicted_label_id = torch.argmax(logits, dim=1).item()
            predicted_label = self.labels[predicted_label_id]
            growth_probability = probabilities[0][1].item()
            adverse_probability = probabilities[0][0].item()

        print("Growth Probability:", growth_probability)
        print("adverse Probability:", adverse_probability)

        return predicted_label

# Example usage
article = article

classifier = RobertaArticleClassifier()
predicted_label = classifier.classify_theme(article)
print("Predicted Label:", predicted_label)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should pr

Growth Probability: 0.5078659653663635
adverse Probability: 0.4921340048313141
Predicted Label: growth


# 3. Bert UnCased

In [24]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

class BertArticleClassifier:
    def __init__(self):
        self.model_name = 'bert-large-uncased'
        self.model = BertForSequenceClassification.from_pretrained(self.model_name)
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.labels = ['adverse', 'growth']
        self.actionable_labels = ['not_actionable', 'actionable']
        self.model.to(self.device)

    def classify_theme(self, article):
        inputs = self.tokenizer.encode_plus(article, return_tensors='pt', truncation=True, padding=True)
        inputs = inputs.to(self.device)

        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            predicted_label_id = torch.argmax(logits, dim=1).item()
            predicted_label = self.labels[predicted_label_id]
            growth_probability = probabilities[0][1].item()
            adverse_probability = probabilities[0][0].item()

        print("Growth Probability:", growth_probability)
        print("Adverse Probability:", adverse_probability)

    def classify_actionable(self, text):
        inputs = self.tokenizer.encode_plus(text, return_tensors='pt', truncation=True, padding=True)
        inputs = inputs.to(self.device)

        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            predicted_label_id = torch.argmax(logits, dim=1).item()
            predicted_label = self.actionable_labels[predicted_label_id]
            not_actionable_probability = probabilities[0][0].item()
            actionable_probability = probabilities[0][1].item()

        print("Not Actionable Probability:", not_actionable_probability)
        print("Actionable Probability:", actionable_probability)

        return predicted_label

In [25]:
classifier = BertArticleClassifier()
predicted_label = classifier.classify_theme(article)
print("Predicted Label:", predicted_label)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly

Growth Probability: 0.3709351420402527
Adverse Probability: 0.6290649175643921
Predicted Label: None


In [26]:
predicted_label = classifier.classify_actionable(article)
print("Predicted Label:", predicted_label)

Not Actionable Probability: 0.6290649175643921
Actionable Probability: 0.3709351420402527
Predicted Label: not_actionable


In [11]:
from transformers import BartTokenizer, BartForConditionalGeneration

In [12]:
class ArticleGistGenerator:
    def __init__(self):
        self.model_name = 'facebook/bart-large-cnn'
        self.tokenizer = BartTokenizer.from_pretrained(self.model_name)
        self.model = BartForConditionalGeneration.from_pretrained(self.model_name)

    def generate_gist(self, article):
        inputs = self.tokenizer([article], max_length=1024, truncation=True, return_tensors='pt')
        summary_ids = self.model.generate(inputs['input_ids'], num_beams=4, max_length=100, early_stopping=True)
        gist = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return gist

In [13]:
classifier = ArticleGistGenerator()
predicted_theme = classifier.generate_gist(article)
print(predicted_theme)

Shares of Bajaj Finance rose as high as 8% to post their biggest single-day gain in nearly one year. The stock surged to near two-year high on Tuesday to close at ₹7,868 apiece on the NSE. Morgan Stanley raised its price target which has the potential to deliver another 18% returns from current levels.


In [40]:
class BertArticleClassifier2:
    def __init__(self):
        self.model_name = 'bert-large-cased'
        self.model = BertForSequenceClassification.from_pretrained(self.model_name)
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        #self.labels = ['adverse', 'growth']
        self.actionable_labels = ['not_actionable', 'actionable']
        self.model.to(self.device)

    # def classify_theme(self, article):
    #     inputs = self.tokenizer.encode_plus(article, return_tensors='pt', truncation=True, padding=True)
    #     inputs = inputs.to(self.device)

    #     with torch.no_grad():
    #         outputs = self.model(**inputs)
    #         logits = outputs.logits
    #         probabilities = torch.softmax(logits, dim=1)
    #         predicted_label_id = torch.argmax(logits, dim=1).item()
    #         predicted_label = self.labels[predicted_label_id]
    #         growth_probability = probabilities[0][1].item()
    #         adverse_probability = probabilities[0][0].item()

    #     print("Growth Probability:", growth_probability)
    #     print("adverse Probability:", adverse_probability)

     #   return predicted_label
    def classify_actionable(self, text):
        inputs = self.tokenizer.encode_plus(text, return_tensors='pt', truncation=True, padding=True)
        inputs = inputs.to(self.device)

        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            predicted_label_id = torch.argmax(logits, dim=1).item()
            predicted_label = self.actionable_labels[predicted_label_id]
            not_actionable_probability = probabilities[0][0].item()
            actionable_probability = probabilities[0][1].item()

        print("Not Actionable Probability:", not_actionable_probability)
        print("Actionable Probability:", actionable_probability)

        return predicted_label

In [41]:
classifier = BertArticleClassifier2()
#predicted_label = classifier.classify_theme(article)
#print("Predicted Label:", predicted_label)

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-cased and are newly ini

In [42]:
predicted_label = classifier.classify_actionable(article)
print("Predicted Label:", predicted_label)

Not Actionable Probability: 0.6418688893318176
Actionable Probability: 0.3581310510635376
Predicted Label: not_actionable


# 5. DistillBert

In [16]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

class DistilBertArticleClassifier:
    def __init__(self):
        self.model_name = 'distilbert-base-uncased'
        self.model = DistilBertForSequenceClassification.from_pretrained(self.model_name)
        self.tokenizer = DistilBertTokenizer.from_pretrained(self.model_name)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.labels = ['adverse', 'growth']
        self.model.to(self.device)

    def classify_theme(self, article):
        inputs = self.tokenizer.encode_plus(article, return_tensors='pt', truncation=True, padding=True)
        inputs = inputs.to(self.device)

        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            predicted_label_id = torch.argmax(logits, dim=1).item()
            predicted_label = self.labels[predicted_label_id]
            growth_probability = probabilities[0][1].item()
            adverse_probability = probabilities[0][0].item()

        print("Growth Probability:", growth_probability)
        print("adverse Probability:", adverse_probability)

        return predicted_label

# Example usage
# article = "Your article text goes here"

classifier = DistilBertArticleClassifier()
predicted_label = classifier.classify_theme(article)
print("Predicted Label:", predicted_label)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.

Growth Probability: 0.4986215829849243
adverse Probability: 0.5013784766197205
Predicted Label: adverse


In [44]:
import torch
from transformers import ElectraTokenizer, ElectraForSequenceClassification

class ArticleClassifier:
    def __init__(self):
        self.model_name = 'google/electra-base-discriminator'
        self.tokenizer = ElectraTokenizer.from_pretrained(self.model_name)
        self.model = ElectraForSequenceClassification.from_pretrained(self.model_name)
        self.label_map = {0: 'Not Growth', 1: 'Growth'}

    def classify(self, article):
        inputs = self.tokenizer.encode_plus(
            article,
            add_special_tokens=True,
            padding='longest',
            truncation=True,
            return_tensors='pt'
        )
        outputs = self.model(**inputs)
        logits = outputs.logits[0]
        probabilities = torch.softmax(logits, dim=0)
        predicted_class = torch.argmax(logits).item()
        predicted_label = self.label_map[predicted_class]
        return predicted_label, probabilities

# Usage
classifier = ArticleClassifier()
#article = "Your article goes here..."
predicted_label, probabilities = classifier.classify(article)
print("Predicted Label:", predicted_label)
print("Growth Probability:", probabilities[1].item())
print("Not Growth Probability:", probabilities[0].item())


Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.o

Predicted Label: Growth
Growth Probability: 0.5368702411651611
Not Growth Probability: 0.46312978863716125


In [45]:
import torch
from transformers import ElectraTokenizer, ElectraForSequenceClassification

class ArticleClassifier:
    def __init__(self):
        self.model_name = 'google/electra-base-discriminator'
        self.tokenizer = ElectraTokenizer.from_pretrained(self.model_name)
        self.model = ElectraForSequenceClassification.from_pretrained(self.model_name)
        self.label_map = {0: 'Not Growth', 1: 'Growth'}
        self.actionable_label_map = {0: 'Not Actionable', 1: 'Actionable'}

    def classify(self, article):
        inputs = self.tokenizer.encode_plus(
            article,
            add_special_tokens=True,
            padding='longest',
            truncation=True,
            return_tensors='pt'
        )
        outputs = self.model(**inputs)
        logits = outputs.logits[0]
        probabilities = torch.softmax(logits, dim=0)
        predicted_class = torch.argmax(logits).item()
        predicted_label = self.label_map[predicted_class]
        return predicted_label, probabilities

    def classify_actionable(self, article):
        predicted_label, probabilities = self.classify(article)
        predicted_class = torch.argmax(probabilities).item()
        actionable_label = self.actionable_label_map[predicted_class]
        return actionable_label, probabilities

# Usage
classifier = ArticleClassifier()
#article = "Your article goes here..."
predicted_label, probabilities = classifier.classify(article)
actionable_label, probabilities = classifier.classify_actionable(article)
print("Growth Classification:")
print("Predicted Label:", predicted_label)
print("Growth Probability:", probabilities[1].item())
print("Not Growth Probability:", probabilities[0].item())

print("\nActionable Classification:")
print("Predicted Label:", actionable_label)
print("Actionable Probability:", probabilities[1].item())
print("Not Actionable Probability:", probabilities[0].item())


Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.o

Growth Classification:
Predicted Label: Not Growth
Growth Probability: 0.49958986043930054
Not Growth Probability: 0.5004101991653442

Actionable Classification:
Predicted Label: Not Actionable
Actionable Probability: 0.49958986043930054
Not Actionable Probability: 0.5004101991653442


In [ ]:
#Use Labels & score to print the output in Seaborn